## SARvey - survey with SAR 

---
<img src="https://seafile.projekt.uni-hannover.de/f/69eac9bb5f86400fa1ad/?dl=1" width="512">


**SARvey** is an open-source software developed for the analysis of Interferometric Synthetic Aperture Radar (InSAR) displacement time series, particularly tailored for engineering applications. It offers a comprehensive workflow to process and analyze Synthetic Aperture Radar (SAR) data, enabling users to monitor and assess ground deformations with high precision.

For more information, visit the following:
* **[SARvey GitHub Repository](https://github.com/luhipi/sarvey)**
* **[SARvey Documentation](https://luhipi.github.io/sarvey/docs/)**
* **[How to cite SARvey](https://sarvey.readthedocs.io/main/readme.html#how-to-cite/)**

---

This tutorial covers the SARvey processing steps for a sample dataset, as a practical example to demonstrate its main capabilities.

---

*This tutorial is prepared by **[Mahmud Haghighi](https://www.ipi.uni-hannover.de/en/haghighi/)** from the Institute of Photogrammetry and GeoInformation, Leibniz University Hannover.*



### SARvey workflow

**TODO: add short description**
<img src="https://seafile.projekt.uni-hannover.de/f/006f702937cd4e618bcb/?dl=1" width="512">




**TODO: add short description**
<img src="https://seafile.projekt.uni-hannover.de/f/39209355cabc4607bf0a/?dl=1" width="512">


### Before we start

This tutorial uses Jupyter Notebook syntax. Here are a few helpful things to know before running the cells:

- `!` at the beginning of a line is used to run shell (terminal) commands directly from the notebook. For example:
    - `!ls` — lists files in the current directory
    - `!pwd` — prints the current working directory
    - `!cd directory_name` — changes the directory (note: cd won't persist across cells unless handled carefully)
    - `!mkdir new_folder` — creates a new directory
    - `!pip install sarvey` — installs the sarvey package
    - `!sarvey -h` — displays the help message for the sarvey command-line interface.

**Note**: You do not need the `!` if you are running these commands directly in a terminal (outside of Notebook).



## Installation

SARvey is a cross-platform python-based software and can be installed on Linux, MacOS and Windows. If you are installing SARvey on your local machine, you can refer to the **[SARvey Installation](https://sarvey.readthedocs.io/main/installation.html#installation)**.

SARvey depends on other open-source software and python packages. Most of dependencies, including *MintPy* are integrated into SARvey installation and will be installed with a simple `pip` command. pip is the package installer for Python. Before installing **SARvey**, we need to install *MiaplPy* separately.

Run the following cell to install **MiaplPy**. This cell will use `pip` for the installation. Note the exclamation mark (!) at the beginning of the command.

In [ ]:
!pip install git+https://github.com/insarlab/MiaplPy.git --quiet

Run the following cell to install **SARvey** along with its required dependencies. These include **MintPy** and several other open-source packages that SARvey relies on to function properly.

In [ ]:
!pip install git+https://github.com/luhipi/sarvey --quiet

After installation, run the following cells to verify that **MiaplPy**, **MintPy** and **SARvey** are installed correctly.

If the installation is successful, you will see the usage prompt for each software.
```
usage: miaplpyApp [-h] [--dir WORKDIR] [-g] [-H] [-v] [--walltime WALL_TIME]


```

If you encounter errors like the one below, it indicates that the software is not installed properly:

```
ERROR: unknown command "miaplpyApp"
```

In [ ]:
!miaplpyApp

In [ ]:
!mintpy -h

In [ ]:
!sarvey -h

## Download data